# Backpropagation example with numpy on MNIST
E. Krupczak - 19 Aug 2018

In [1]:
import numpy as np

Last node goes to loss, so d(loss)/d(final node) = 1
$$\frac{d(loss)}{d(node)} = \text{node weight}$$
i.e. for an affine layer:
$$Loss = x_1+x_2+x_3+x_4$$
$$\frac{dLoss}{dx_1} = 1$$
and for a non-linear layer:
$$Loss = \tanh(x_1)$$
$$\frac{dLoss}{dx_1}= \frac{d\tanh(x_1)}{dx_1}$$

Use chain rule to combine layers, eg. $\alpha$, $\beta$:
$$\frac{dL}{d\alpha} = \frac{d\beta}{d\alpha}\cdot\frac{dL}{d\beta}$$

Class for every kind of layer:
- Affine layer
    - Matrix multiplication layer
    - Bias layer
- Nonlinear layer (eg. tanh)
- Loss/squared-error layer
- Softmax cross entropy layer